In [2]:
import numpy as np
from event_yields import calculate_event_yields


In [3]:
muons_on_target = [1e18, 1e20, 1e22]
room_temp = 293.1
GeV_to_J = 1.602e-10
hbarc = .197  # GeV fm
hbarc_cm_invGeV = hbarc * 1e-13

E_0s = [10, 63, 1500, 5000]
targets = ["Water", "Lead"]

target_lengths_water = {10: 4, 63: 10, 1500: 10, 5000: 10}
target_lengths_lead = {10: 0.5, 63: 2, 1500: 5, 5000: 5}

l_T = {"Water" : target_lengths_water, "Lead" :  target_lengths_lead}
Zs = {"Water" : 10, "Lead" : 82}
As = {"Water" : 18.02, "Lead" : 207.2}
rhos = {"Water" : 1.0, "Lead" : 11.35}

In [8]:
# Boiling / melting
critical_temp = {
                "Water" : 373.1,
                "Lead" : 600.6,
                }


# Heat Capacity [J/g*mol*K]
# NOTE: Checked roughly constant in temperature range of interet [300-600 K]
heat_capacity = {
                "Water" : 4.18,
                "Lead" : 0.129,
                }


def total_cross_section(energy, material):

    # Load config and cross sections
    config = np.load(f"experiments/{material}/cross_sections/config_{energy}.npy", allow_pickle=True)[()]

    run_IWW = config["IWW"]
    run_WW = config["WW"]
    cross_section_directory = config["Cross Sections Directory"]
    cross_section_file = config["Cross Sections File"]

    if run_IWW:
        cross_section_dict = np.load(f"{cross_section_file}.npy", allow_pickle=True)[()]
    if run_WW:
        cross_section_dict = np.load(f"{cross_section_file}_WW.npy", allow_pickle=True)[()]

    xbins = config["xbins"]
    xs = np.linspace(0, 1, xbins)
    m_Xs = np.array(config["m_X"])


    mass_cross_sections = (hbarc_cm_invGeV)**2 * np.array([np.nansum((1 - xs) * cross_section_dict["Vector"][i]/ xbins, ) for (i, m_X) in enumerate(m_Xs[m_Xs < energy])])
    return np.max(mass_cross_sections)


def effective_mass(energy, material):

    cross_section = total_cross_section(energy, material)
    interaction_volume = l_T[material][energy] * 100 * cross_section * 6e23
    interaction_mass = rhos[material] * interaction_volume
    return interaction_mass / As[material]


print(effective_mass(63, "Lead"))


def one_year_energy(energy, muons, material):
        return energy * muons * GeV_to_J  #* effective_mass(energy, material)



# def one_year_temp_change(energy, muons, material):

#     return energy * muons * GeV_to_J / heat_capacity[material] * effective_mass(energy, material)

for m in targets:
    for e in E_0s:
        for n in [1e20]:
            
            print(e, "&",  one_year_energy(e, n, "Water") / 1e9, "&", one_year_energy(e, n, "Water") / (3.14e7) / 1000, "&", one_year_energy(e, n, "Lead") / 1e9 , "&", one_year_energy(e, n, "Lead") / (3.14e7) / 1000, "\\\\")
            # print(m, e, n, one_year_energy(e, n, m), "Joules --> ", one_year_energy(e, n, m) / (3.14e7), "Watts")

0.07145246984031264
10 & 160.2 & 5.101910828025478 & 160.2 & 5.101910828025478 \\
63 & 1009.26 & 32.14203821656051 & 1009.26 & 32.14203821656051 \\
1500 & 24030.0 & 765.2866242038216 & 24030.0 & 765.2866242038216 \\
5000 & 80100.0 & 2550.955414012739 & 80100.0 & 2550.955414012739 \\
10 & 160.2 & 5.101910828025478 & 160.2 & 5.101910828025478 \\
63 & 1009.26 & 32.14203821656051 & 1009.26 & 32.14203821656051 \\
1500 & 24030.0 & 765.2866242038216 & 24030.0 & 765.2866242038216 \\
5000 & 80100.0 & 2550.955414012739 & 80100.0 & 2550.955414012739 \\


The formula for the total energy deposit is:

$\Delta E = (\int dx N_\mu \frac{\rho L_{tar}}{m_T} \frac{d\sigma}{dx}(1-x)) \times E["0"]$

In one sense, this formula is an overestimate, since even though only $(1-x)$ of energy is left after an $X$ emission, this energy is split between the nucleus and muon.

On the other hand, this formula is an underestimate, since $\sigma_{tot}$ should in principle include ALL modes of interaction (including/especially DIS?) between a muon and a nucleus, but here we've only included induced bremstraulung. These other interaction modes would likely affect the nuclei a lot more violently




The formula for the total change in material temperature is:

$\Delta T =  (N_\mu \frac{\rho L_{tar} \sigma_{tot}}{m_T})   \times \frac{E["0"]}{\rho (L_{tar}\sigma_{tot}) C_v}$

where $\sigma_{tot}$ is the total interaction cross section -- here, I take it to be the maximum cross section across all possible masses and spins of $X$.

In one sense, this formula is a huge overestimate, since even for muons that interact with nuclei, only a small fraction of $E["0"]$ is actually imparted onto the nucleus, as most of it just goes into $X$

On the other hand, this formula is an underestimate, since $\sigma_{tot}$ should in principle include ALL modes of interaction (including/especially DIS?) between a muon and a nucleus, but here we've only included induced bremstraulung. These other interaction modes would likely affect the nuclei a lot more violently

In [20]:
aw = {}
bw = {}
al = {}
bl = {}




aw["3"], bw["3"] = 2.287 , 1e-6 
aw["10"], bw["10"] = 2.482 , 1.4380e-6 
aw["100"], bw["100"] = 2.781 , 2.278e-6 
aw["1000"], bw["1000"] = 3.325 , 2.9575e-6 
aw["10000"], bw["10000"] = 3.634 , 3.4961e-6 

al["3"], bl["3"] = 1.442 , 3e-6 
al["10"], bl["10"] = 1.615 , 6.7899e-6 
al["100"], bl["100"] = 1.860 , 12.6448e-6
al["1000"], bl["1000"] = 2.051 , 16.4724e-6 
al["10000"], bl["10000"] = 2.251 , 18.3613e-6

at = {"Water": aw, "Lead" : al}
bt = {"Water": bw, "Lead" : bl}

def energy_loss(E, n, material):

    L = l_T[material][E]
    E_mev = E * 1000
    L_cm = L * 100

    a = np.interp(E, [*at[material].keys()], [*at[material].values()])
    b = np.interp(E, [*bt[material].keys()], [*bt[material].values()])

    x = np.exp(-L_cm * b) * (a + b * E_mev) - a
    x = x / (b * E_mev)
    return (1-x) * E * GeV_to_J


for m in targets:
    for e in E_0s:
        for n in [1e20]:
            
            print(e, "&",  "%.2f" % (n*energy_loss(e, n, "Water") / 1e9), "&", "%.2f" % (n*energy_loss(e, n, "Water") / (3.14e7) / 1000), "&","%.2f" % (n*energy_loss(e, n, "Lead") / 1e9) , "&", "%.2f" % (n*energy_loss(e, n, "Lead") / (3.14e7) / 1000), "\\\\")


10 & 15.99 & 0.51 & 1.35 & 0.04 \\
63 & 44.49 & 1.42 & 7.70 & 0.25 \\
1500 & 125.14 & 3.99 & 214.80 & 6.84 \\
5000 & 311.04 & 9.91 & 707.42 & 22.53 \\
10 & 15.99 & 0.51 & 1.35 & 0.04 \\
63 & 44.49 & 1.42 & 7.70 & 0.25 \\
1500 & 125.14 & 3.99 & 214.80 & 6.84 \\
5000 & 311.04 & 9.91 & 707.42 & 22.53 \\
